# Prepación de los Datos

En la notebook anterior, hicimos el análisis exploratorio de los datos y los dividimos en conjunto de entrenamiento y prueba.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd '/content/drive/MyDrive/Inteligencia Artificial/IA - Clases de Práctica/ContenidosPorTemas'
    print('google.colab')

In [ ]:
np.random.seed(42)

# Cargamos los datos
df_housing = pd.read_csv("./1_datos/housing.csv")

# Creamos nuestro atributo categórico para los ingresos
df_housing["income_cat"] = pd.cut(df_housing["median_income"], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5])

# Dividimos los datos en conjunto de entrenamiento y prueba
train_set, test_set = train_test_split(df_housing, test_size=0.2, stratify=df_housing["income_cat"], random_state=42)

# Eliminamos la categoria income_cat de ambos conjuntos porque no la usamos
for set_ in (train_set, test_set):
    set_.drop("income_cat", axis=1, inplace=True)

## Separamos los predictores de las etiquetas

In [ ]:
housing = train_set.drop("median_house_value", axis=1)
housing_labels = train_set["median_house_value"].copy()

## Limpieza de Datos

La mayoría de los algoritmos de aprendizaje automático no pueden trabajar con características faltantes, por lo que debemos ocuparnos de esto. Por ejemplo, vimos que el atributo `total_bedrooms` tiene algunos valores faltantes. Existen tres opciones para solucionar esto:

1. Eliminar los distritos correspondientes.
2. Eliminar el atributo.
3. Reemplazar los valores faltantes con algún valor (cero, la media, la mediana, etc.). A esto se llama _imputación_.

In [ ]:
# índices de las filas con datos faltantes
null_rows_idx = housing.isnull().any(axis=1)
housing.loc[null_rows_idx].head()

In [ ]:
housing.loc[null_rows_idx].info()

In [ ]:
housing_option1 = housing.copy()

housing_option1.dropna(subset=["total_bedrooms"], inplace=True)  # opción 1: elimino los distritos con datos faltantes

housing_option1.loc[null_rows_idx].head()

In [ ]:
housing_option2 = housing.copy()

housing_option2.drop("total_bedrooms", axis=1, inplace=True)  # opción 2: elimino la columna

housing_option2.loc[null_rows_idx].head()

In [ ]:
housing_option3 = housing.copy()

median = housing["total_bedrooms"].median()
housing_option3["total_bedrooms"].fillna(median, inplace=True)  # opción 3: reemplazo los valores faltantes con la mediana

housing_option3.loc[null_rows_idx].head()

vamos a optar por la opción 3 ya que es la menos destructiva, pero en lugar del código anterior, usaremos una clase de Scikit-Learn:   `SimpleImputer`.

La ventaja que esto posee es que almacenará el valor de la mediana de cada característica. Esto permitirá imputar valores faltantes no solo en el conjunto de entrenamiento, sino también en el conjunto de pruebas y cualquier dato nuevo que ingrese al modelo.

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

La mediana solo puede calcularse en atributos numéricos, creamos una copia de los datos solo con atributos numéricos (excluyendo `ocean_proximity`)

In [ ]:
housing_num = housing.drop("ocean_proximity", axis=1)
imputer.fit(housing_num)

In [ ]:
imputer.statistics_

In [ ]:
housing_num.median(numeric_only=True).values


Ahora se puede usar este imputador "entrenado" para transformar el conjunto de entrenamiento reemplazando los valores faltantes con las medianas calculadas:

In [ ]:
X = imputer.transform(housing_num) # me devuelve un numpy array
type(X)

Los valores faltantes también pueden reemplazarse con el valor medio `(strategy="mean")`, o con el valor más frecuente `(strategy="most_frequent")`, o con un valor constante `(strategy="constant", fill_value=...)`. Las dos últimas estrategias admiten datos no numéricos.

In [ ]:
imputer.feature_names_in_

In [ ]:
housing_train = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)
housing_train.loc[null_rows_idx].head()

## Manejo de atributos categóricos

Nuestro conjunto de datos tiene el atributo `ocean_proximity` que es de tipo categórico. La mayoría de algoritmos de aprendizaje maquinal no trabaja con atributos numéricos, por lo que debemos convertir estas categorías de tipo texto a un valor numérico.

In [ ]:
housing_cat = housing.loc[:,["ocean_proximity"]]
housing_cat.head(8)

Primero probaremos la clase `OrdinalEncoder` de Scikit-Learn

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)

In [ ]:
housing_cat_encoded[:8]

In [ ]:
ordinal_encoder.categories_

Un problema con esta representación es que los algoritmos de aprendizaje maquinal asumirán que dos valores cercanos son más similares que dos valores distantes. Esto puede ser aceptable en algunos casos (por ejemplo, para categorías ordenadas como "malo", "regular", "bueno" y "excelente"), pero no es el caso para la columna ocean_proximity (por ejemplo, las categorías 0 y 4 son claramente más similares que las categorías 0 y 1).

Para solucionar este problema, una solución común es crear un atributo binario por categoría: un atributo igual a 1 cuando la categoría es `<1H OCEAN` (y 0 en caso contrario), otro atributo igual a 1 cuando la categoría es `INLAND` (y 0 en caso contrario), y así sucesivamente. Esto se llama codificación _one-hot_, porque solo un atributo será igual a 1 (activo), mientras que los demás serán 0 (inactivo). Los nuevos atributos a veces se llaman atributos ficticios (atributos _dummy_). Scikit-Learn proporciona la clase OneHotEncoder para convertir valores categóricos en vectores one-hot.

Clase `OneHotEncoder` de Scikit-Learn

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

Por defecto, la salida de un `OneHotEncoder` es una matriz rala (sparse matrix), en lugar de un arreglo NumPy. Una matriz rala es una representación muy eficiente para matrices que contienen principalmente ceros. Internamente, solo almacena los valores no nulos y sus posiciones. Cuando un atributo categórico tiene cientos o miles de categorías, codificarlo con one-hot resulta en una matriz muy grande llena de 0s excepto por un solo 1 por fila. Una matriz rala ahorra mucha memoria y acelerará los cálculos.

Se puede usar una matriz rala de manera similar a un arreglo 2D normal, pero se puede convertir a un arreglo NumPy (denso) con el método `toarray()`. También puede fijarse `sparse=False` al crear el `OneHotEncoder`, en cuyo caso el método `transform()` devolverá directamente un arreglo NumPy.

In [ ]:
housing_cat_1hot.toarray()

Cuando entrenamos cualquier estimador de Scikit-Learn utilizando un DataFrame, el estimador guarda los nombres de las columnas en el atributo `feature_names_in_`. Luego, Scikit-Learn asegura que cualquier DataFrame proporcionado a este estimador después de de ser entrenado (por ejemplo, a transform() o predict()) tenga los mismos nombres de columnas. También proporcionan un método `get_feature_names_out()` que se puede usar para construir un DataFrame:

In [ ]:
cat_encoder.feature_names_in_

In [ ]:
cat_encoder.get_feature_names_out()

In [ ]:
df_housing_cat_1hot = pd.DataFrame(housing_cat_1hot.toarray(),
                         columns=cat_encoder.get_feature_names_out(),
                         index=housing_cat.index)

df_housing_cat_1hot

## Transformación y escalado de características

Una de las transformaciones más importantes es el _escalado de características_ (feature scaling). Con pocas excepciones, los algoritmos de aprendizaje maquinal no resultan en un buen desempeño cuando los atributos numéricos de entrada tienen escalas muy diferentes. En nuestro caso: el número total de habitaciones varía entre 6 a 39,320, los ingresos solo varían desde 0 hasta 15. Sin escalar previamente los datos, el atributo `total_rooms` tendrá una influencia dominante en el modelo de aprendizaje automático ignorando el atributo `median_income`.

Existen dos formas: el escalado _min-max_ y la _estandarización_.

### Escalado min-max

Se lo llama comunmente normalización. Para cada atributo, los valores se escalan en el rango entre 0 a 1. Esto se logra restando el valor mínimo y dividiendo por la diferencia entre el mínimo y el máximo.
Scikit-Learn tiene un _transformer_ llamado `MinMaxScaler` que posee un hiperparámetro `feature_range` para cambiar el rango si no se quiere de 0 a 1 (por ejemplo, las redes neuronales funcionan mejor con entradas de media cero, por lo que un rango de -1 a 1 es preferible).

In [ ]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
housing_num_min_max_scaled = min_max_scaler.fit_transform(housing_num)

In [ ]:
housing_num_min_max_scaled

### Estandarización

Primero se resta el valor medio (los valores estandarizados tienen media de cero), luego divide el resultado por el desvío estándar (los valores tienen un desvío estándar igual a 1). A diferencia del escalado min-max, la estandarización no restringe los valores a un rango específico. La estandarización se ve mucho menos afectada por los valores atípicos.

Scikit-Learn proporciona un _transformer_ llamado `StandardScaler`:

In [ ]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
housing_num_std_scaled = std_scaler.fit_transform(housing_num)

In [ ]:
housing_num_std_scaled

✅ Si se quiere escalar una matriz rala sin convertirla en una matriz densa, se puede utilizar `StandardScaler` con el hiperparámetro `with_mean` igual a `False`. Esto solo dividirá los datos por el desvío estándar, sin restar la media (sino dejaría de ser rala):

Cuando la distribución de una característica está muy sesgada, tanto el escalado min-max como la estandarización comprimirán la mayoría de los valores en un rango pequeño. Por lo tanto, antes de escalar un atributo, se debería transformar para reducir el sesgo y, si es posible, hacer que la distribución sea aproximadamente simétrica (normal).

Una forma común de hacer esto para características positivas con sesgo hacia la derecha es reemplazar la característica por su raíz cuadrada (o elevar la característica a una potencia entre 0 y 1).

Si la característica tiene una cola muy larga y pesada (sesgo), como una distribución de potencias, se puede reemplazar la característica por su logaritmo.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 7), sharey=True)

housing["population"].hist(ax=axs[0], bins=50)
housing["population"].apply(np.log).hist(ax=axs[1], bins=50)

axs[0].set_xlabel("Population")
axs[1].set_xlabel("Log of population")
axs[0].set_ylabel("Number of districts")

plt.show()

Otro enfoque consiste en dividir la característica en "buckets" o intervalos. Esto implica dividir su distribución en intervalos de tamaño aproximadamente igual y reemplazar cada valor de la característica con el índice del intervalo al que pertenece. Por ejemplo, podríamos reemplazar cada valor con su percentil. Esto resulta en una característica con una distribución casi uniforme, por lo que no es necesario realizar escalado, o simplemente se puede dividir por el número de intervalos para forzar que los valores estén en el rango de 0 a 1.

In [ ]:
np.percentile(housing["median_income"], 50)

In [ ]:
percentiles = [ np.percentile(housing["median_income"], p) for p in range(1, 100) ]

In [ ]:
flattened_median_income = pd.cut(housing["median_income"],
                                 bins=[-np.inf] + percentiles + [np.inf],
                                 labels=range(1, 100 + 1))

flattened_median_income.hist(bins=50, figsize=(10,7))
plt.xlabel("Median income percentile")
plt.ylabel("Number of districts")
plt.show()

Hasta ahora, solo hemos analizado las características de entrada, pero es posible que también sea necesario transformar los valores target. Por ejemplo, si la distribución del target tiene un sesgo, podemos aplicarle el logaritmo. Sin embargo, un modelo de regresión predecirá el logaritmo del `median_income`, no el valor en sí. Deberíamos calcular la exponencial de la predicción para obtener el valor buscado.

## Resumen de la segunda parte: Preparación de los datos

En esta segunda parte:
- Hicimos una limpieza de los datos de entrenamiento:
    - Manejo de datos faltantes o erróneos. Imputamos valores (media, mediana, etc)
- Manejo de atributos categóricos
    - OrdinalEncoder
    - Codificación OneHot
- Transformación y escalado de atributos